<a href="https://colab.research.google.com/github/dkapitan/jads-nhs-proms/blob/master/notebooks/3.0-data-preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background to osteoarthritis case study

_taken from [narrative seminar Osteoarthritis by Hunter & Bierma-Zeinstra (2019) in the Lancet](https://github.com/dkapitan/jads-nhs-proms/blob/master/references/hunter2019osteaoarthritis.pdf)._

Outcomes from total joint replacement can be optimised if patient selection identifies marked joint space narrowing. Most improvement will be made in patients with complete joint space loss and evident bone attrition. Up to 25% of patients presenting for total joint replacement continue to complain of pain and disability 1 year after well performed surgery. Careful preoperative patient selection (including consideration of the poor outcomes that are more common in people who are depressed, have minimal radiographic disease, have minimal pain, and who are morbidly obese), shared decision making about surgery, and informing patients about realistic outcomes of surgery are needed to minimise the likelihood of dissatisfaction.

# Data Preparation

This is day 2 from the [5-day JADS NHS PROMs data science case study](https://github.com/dkapitan/jads-nhs-proms/blob/master/references/outline.md).



## Learning objectives: select and clean data
- Impute missing values
- Select main input variables X (feature engineering)
- Define target Y (clustered classes, categories)
- Decide how to handle correlated input features

## Learning objectives: Python

- [Pythonic data cleaning](https://realpython.com/python-data-cleaning-numpy-pandas/)
- [When to use list comprehensions](https://realpython.com/list-comprehension-python/)
- [Python f-strings](https://realpython.com/python-f-strings/)
- [Pandas GroupBy](https://realpython.com/pandas-groupby/)
- [Correlations in with numpy, scipy and pandas](https://realpython.com/numpy-scipy-pandas-correlation-python/)
- Extra: [Using itertools](https://realpython.com/python-itertools/#what-is-itertools-and-why-should-you-use-it)

## Recap from previous lecture
- Good outcome for knee replacement Y is measured using difference in Oxford Knee Score (OKS)
- Research has shown that an improvement in OKS score of approx. 30% is relevant ([van der Wees 2017](https://github.com/dkapitan/jads-nhs-proms/blob/master/references/vanderwees2017patient-reported.pdf)). Hence an increase of +14 points is considered a 'good' outcome.
- to account for the ceiling effect, a high final `t1_oks_score` is also considered as a good outcome (even if `delta_oks_score` is smaller than 14).

    

# Select and clean data


In [0]:
import warnings
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.feature_selection import chi2, VarianceThreshold
import sklearn.linear_model

#supressing warnings for readability
warnings.filterwarnings("ignore")

# To plot pretty figures directly within Jupyter
%matplotlib inline

# choose your own style: https://matplotlib.org/3.1.0/gallery/style_sheets/style_sheets_reference.html
plt.style.use('ggplot')

# Go to town with https://matplotlib.org/tutorials/introductory/customizing.html
# plt.rcParams.keys()
mpl.rc('axes', labelsize=14, titlesize=14)
mpl.rc('figure', titlesize=20)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# contants for figsize
S = (8,8)
M = (12,12)
L = (14,14)

# pandas options
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)
pd.set_option("display.precision", 2)

# import data
df = pd.read_parquet('https://github.com/dkapitan/jads-nhs-proms/blob/master/data/interim/knee-provider.parquet?raw=true')

In [0]:
# re-run relevant code from previous lecture
# replace sentinel values with NA
dfc = df.copy()
dfc.loc[:,['t0_eq_vas', 't1_eq_vas']] = dfc.loc[:,['t0_eq_vas', 't1_eq_vas']].replace(999, np.nan).astype('Int64')
_no9 = [col for col in df.columns if col.startswith('oks_t') and not col.endswith('score')]
dfc.loc[:,_no9] = df.loc[:,_no9].replace(9, np.nan).astype('Int64')

# add delta_oks_score and Y
def good_outcome(oks_t1, delta_oks, abs_threshold=43, mcid=13):
  if oks_t1 > abs_threshold or delta_oks > mcid:
    return True
  else:
    return False


dfc['delta_oks_score'] = dfc.oks_t1_score - dfc.oks_t0_score
dfc['Y'] = dfc.apply(lambda row: good_outcome(row['oks_t1_score'], row['delta_oks_score']), axis=1)

## Clean X


### Intermezzo: Python skills



#### Using list comprehensions

There are three ways to work with lists in Python:
1. with `for` loops
2. using `map` (map-reduce pattern)
3. using list comprehensions

The third option finds its roots in functional programming and is considered the most Pythonic. It is particularly useful for data science.

In this case, we want to make group of columns by name, without having to continously re-type the names. We will use some functions from the [itertools](https://docs.python.org/3.7/library/itertools.html#module-itertools) standard library.

In [88]:
from itertools import product

# a product generates a carterian product
['_'.join(col) for col in product(['t0', 't1'],  ['assisted', 'previous_surgery', 'disability'])]

['t0_assisted',
 't0_previous_surgery',
 't0_disability',
 't1_assisted',
 't1_previous_surgery',
 't1_disability']

In [0]:
# helper function to count top 10 unique values for dataframe
# pd.concat takes as list of Series or DataFrames to concatenate
# make this list using a list comprehension
def count_values(df):
  _value_counts = [df[col].value_counts().sort_values(ascending=False).head(10) for col in df.columns]
  return pd.concat(_value_counts, axis=1).transpose()

#### Using f-strings

In [94]:
def oks_questions(t='t0'):
  return [f'oks_{t}_pain', f'oks_{t}_night_pain', f'oks_{t}_washing',
          f'oks_{t}_transport', f'oks_{t}_walking', f'oks_{t}_standing',
          f'oks_{t}_limping', f'oks_{t}_kneeling', f'oks_{t}_work',
          f'oks_{t}_confidence', f'oks_{t}_shopping', f'oks_{t}_stairs', ]

def eq5d_questions(t='t0'):
  return [f'{t}_mobility', f'{t}_self_care',
          f'{t}_activity', f'{t}_discomfort', f'{t}_anxiety']

oks_questions()

['oks_t0_pain',
 'oks_t0_night_pain',
 'oks_t0_washing',
 'oks_t0_transport',
 'oks_t0_walking',
 'oks_t0_standing',
 'oks_t0_limping',
 'oks_t0_kneeling',
 'oks_t0_work',
 'oks_t0_confidence',
 'oks_t0_shopping',
 'oks_t0_stairs']

### Inspect boolean features

In [89]:
df.columns

Index(['provider_code', 'procedure', 'revision_flag', 'year', 'age_band',
       'gender', 't0_assisted', 't0_assisted_by', 't0_symptom_period',
       't0_previous_surgery', 't0_living_arrangements', 't0_disability',
       'heart_disease', 'high_bp', 'stroke', 'circulation', 'lung_disease',
       'diabetes', 'kidney_disease', 'nervous_system', 'liver_disease',
       'cancer', 'depression', 'arthritis', 't0_mobility', 't0_self_care',
       't0_activity', 't0_discomfort', 't0_anxiety', 't0_eq5d_index_profile',
       't0_eq5d_index', 't1_assisted', 't1_assisted_by',
       't1_living_arrangements', 't1_disability', 't1_mobility',
       't1_self_care', 't1_activity', 't1_discomfort', 't1_anxiety',
       't1_satisfaction', 't1_sucess', 't1_allergy', 't1_bleeding', 't1_wound',
       't1_urine', 't1_further_surgery', 't1_readmitted',
       't1_eq5d_index_profile', 't1_eq5d_index',
       'oks_eq_5d_index_t1_predicted', 't0_eq_vas', 't1_eq_vas',
       'oks_eq_vas_t1_predicted', 'oks

In [74]:
casemix = ['heart_disease', 'high_bp', 'stroke', 'circulation', 'lung_disease', 'diabetes',
           'kidney_disease', 'nervous_system', 'liver_disease', 'cancer', 'depression', 'arthritis']
boolean = ['t0_assisted', 't0_previous_surgery', 't0_disability']
count_values(df[['gender'] + casemix + boolean])

,1.0,2.0,9.0
gender,55749.0,74085.0,NaN
heart_disease,13051.0,NaN,126185.0
high_bp,61570.0,NaN,77666.0
stroke,2303.0,NaN,136933.0
circulation,7882.0,NaN,131354.0
lung_disease,12779.0,NaN,126457.0
diabetes,17379.0,NaN,121857.0
kidney_disease,2856.0,NaN,136380.0
nervous_system,1422.0,NaN,137814.0
liver_disease,812.0,NaN,138424.0


### Inspect categorical features

In [79]:
eq5d = ['t0_mobility', 't0_self_care', 't0_activity', 't0_discomfort', 't0_anxiety']
categorical = ['t0_symptom_period', 't0_previous_surgery', 't0_living_arrangements']

# useless variables
useless = ['t0_assisted_by', 't0_eq5d_index', 't0_eq5d_index_profile']

count_values(df[eq5d + categorical])

,1,2,3,4,9
t0_mobility,10799.0,123763.0,388.0,NaN,4286.0
t0_self_care,93976.0,39843.0,1007.0,NaN,4410.0
t0_activity,12648.0,105171.0,16928.0,NaN,4489.0
t0_discomfort,1408.0,80643.0,51645.0,NaN,5540.0
t0_anxiety,85417.0,43307.0,5351.0,NaN,5161.0
t0_symptom_period,6488.0,71767.0,30434.0,29338.0,1209.0
t0_previous_surgery,10784.0,127410.0,NaN,NaN,1042.0
t0_living_arrangements,106041.0,30489.0,133.0,486.0,2087.0


## Impute missing values

### Strategy for missing values

* No missing values: gender, age_band and all casemix variables
* Booleans with missing values: most frequent
* Categorical:
  * eq5d: use most frequent
  * symptom_period, living arrangements: use most frequent
* Numerical: impute EQ-VAS with median

Since we are going to replace values, we choose to convert the data to the most optimal type.

Write code in such a way that you can easily adopt this strategy.

In [82]:
from sklearn.impute import SimpleImputer

impute_median = SimpleImputer(strategy='median')
impute_most_frequent = SimpleImputer(strategy='most_frequent')

# use copy of original data
dfc = df.copy()

# clean features with 9 sentinel values
_no9 = boolean + eq5d + categorical
dfc.loc[:,_no9] = df.loc[:,_no9].replace(9, np.nan)
impute_most_frequent.fit(dfc[_no9])
impute_most_frequent.statistics_

array([2., 2., 1., 2., 1., 2., 2., 1., 2., 2., 1.])

In [0]:
# clean features with 999 sentinel values
dfc.loc[:,['t0_eq_vas', 't1_eq_vas']] = dfc.loc[:,['t0_eq_vas', 't1_eq_vas']].replace(999, np.nan)

# impute with most frequent values


# features with no missing values
dfc.loc[:,'gender'] = dfc.loc[:,'gender'].replace({1: False, 2: True})
dfc.loc[:, casemix] = dfc.loc[:, casemix].replace({9: False, 1: True})


In [0]:


# helper function for counting boolean attribues
def count_boolean(series):
    '''
    Returns absolute and normalized value counts of pd.series as a dataframe with 
    index = series.name
    columns with absolute and normalized counts of each value
    '''
    try:
        count = series.value_counts().to_frame().transpose()
        norm = series.value_counts(normalize=True).to_frame().transpose()
        return count.join(norm, lsuffix='_count', rsuffix='_normalized') 
    except:
        print('Error: expecting a pandas.Series object as input. \n' + count_all.__doc__)
        return None

In [65]:
tbl_casemix = pd.concat([count_all(dfc.loc[:, feature]) for feature in casemix], sort=True)
tbl_casemix.round(2).sort_values('True_normalized', ascending=False)

,False_count,False_normalized,True_count,True_normalized
arthritis,31962,0.23,107274,0.77
high_bp,77666,0.56,61570,0.44
diabetes,121857,0.88,17379,0.12
heart_disease,126185,0.91,13051,0.09
lung_disease,126457,0.91,12779,0.09
depression,126205,0.91,13031,0.09
circulation,131354,0.94,7882,0.06
cancer,131693,0.95,7543,0.05
stroke,136933,0.98,2303,0.02
kidney_disease,136380,0.98,2856,0.02


## Discussion

### **Question:** what are relevant considerations to handle NAs?
- imputation with mean/median?
- just drop all?

### **Question:** what would you choose as the primary outcome Y?

# Conclusion and reflection

## Discussion of results

* ...
* ...


## Checklist for results from data preparation process
* ...
* ...
